In [1]:
%run base_test_regressors.ipynb

In [2]:
type_ = 'grid'
average = True
poly = True
bilinear = True

results = test_regressors(type_=type_, average=average, poly=poly, 
                          bilinear=bilinear)